In [101]:
# read a yaml file of reactions, check atom balance
import yaml, sys, os
sys.path.insert(0, '../../karlib')
import chem_subs as chem
import molpro_subs as pro
import gaussian_subs as gau
import multirx_subs as mrx

In [102]:
def read_yaml(file):
    # read a yaml file, return the contents
    with open(file, 'r') as Y:
        x = yaml.safe_load(Y)
    return x
def read_molec_yaml(molec):
    # read a molecule's yaml data file
    f = os.sep.join(['molec_data', f'{molec}.yml'])
    return read_yaml(f)
def build_Geom(molec):
    # create a Geometry() object using data from global dict 'moldata'
    coord = moldata[molec]['Geometry']['coordinates']
    G = chem.Geometry(coord)
    return G

In [103]:
x = read_yaml('reactions.yml')
target = x['target']
rxns = x['reactions']

In [104]:
print(f'{len(rxns)} reactions for {target}')

15 reactions for dioxolane


In [105]:
# read molecular yaml files in subdirectory 'molec_data'
moldata = {}
Geom = {}
missing = []  # list of educts without molecular yaml files
for rxn in rxns:
    for pair in rxn:
        educt = pair[0]
        if educt not in moldata.keys():
            try:
                moldata[educt] = read_molec_yaml(educt)
                Geom[educt] = build_Geom(educt)
            except:
                # assume that the problem is a missing yaml file
                missing.append(educt)
if missing:
    print('*** Missing YAML files in molec_data/ ***')
    for m in missing:
        print(f'\t{m}')

In [106]:
def reaction_imbalance(Glist, coeffs):
    # given a list of Geometry() and of numerical multipliers,
    # return a dict of net changes in atom (element) number
    counts = {}
    for G, coeff in zip(Glist, coeffs):
        for k, v in G.stoichiometry(asdict=True).items():
            if k in counts.keys():
                counts[k] += v * coeff
            else:
                counts[k] = v * coeff
    # don't include atoms with zero net counts
    imbal = {k: v for k, v in counts.items() if v != 0}
    return imbal

In [107]:
for i, rxn in enumerate(rxns):
    Glist = []
    clist = []
    for pair in rxn:
        educt = pair[0]
        Glist.append(Geom[educt]) # educt name
        clist.append(pair[1]) # multiplier
    rnet = reaction_imbalance(Glist, clist)
    if rnet:
        print('*** Imbalance of', chem.stoichiometry(rnet, ones=False))
        print('for reaction', rxn)
    else:
        print(f'Reaction {i} is balanced:', rxn)

Reaction 0 is balanced: [['dioxolane', 1], ['n2', 3], ['hcn', -3], ['no', -2], ['nh3', -1]]
Reaction 1 is balanced: [['dioxolane', 1], ['hcl', 5], ['ch3cl', -3], ['hocl', -2]]
Reaction 2 is balanced: [['dioxolane', 1], ['h2', 5], ['ch4', -3], ['h2o', -2]]
Reaction 3 is balanced: [['dioxolane', 1], ['meoac', -1]]
Reaction 4 is balanced: [['dioxolane', 1], ['o2', 3.5], ['co2', -3], ['h2o', -3]]
Reaction 5 is balanced: [['dioxolane', 1], ['cyc5', 1], ['THF', -2]]
Reaction 6 is balanced: [['dioxolane', 1], ['ch2o', -1], ['oxirane', -1]]
Reaction 7 is balanced: [['dioxolane', 1], ['h2o', 1], ['ch2o', -3], ['h2', -1]]
Reaction 8 is balanced: [['dioxolane', 1], ['acetone', 1], ['EC', -1], ['c3h8', -1]]
Reaction 9 is balanced: [['dioxolane', 1], ['c3h8', 2], ['cyc5', -1], ['DME', -2]]
Reaction 10 is balanced: [['dioxolane', 1], ['o2', 2], ['co', -3], ['h2o', -3]]
Reaction 11 is balanced: [['dioxolane', 1], ['ch4', 3], ['h2o', 2], ['c2h6', -1], ['ch3oh', -4]]
Reaction 12 is balanced: [['dioxola